## Number of Shipments Per Month (nsm)

Write a query that will calculate the number of shipments per month. The unique key for one shipment is a combination of shipment_id and sub_id. Output the year_month in format YYYY-MM and the number of shipments in that month.

Table: amazon_shipment_nsm

In [1]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import os

In [3]:
os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-11"

spark = SparkSession.builder.appName('NSM').master('local').getOrCreate()

In [6]:
df = spark.read.format('csv').\
        option('header', True).\
        option('inferSchema', True).\
        load('../Data/amazon_shipment_nsm.csv')
        
df.show(5)

+-----------+------+------+-------------+
|shipment_id|sub_id|weight|shipment_date|
+-----------+------+------+-------------+
|        101|     1|    10|   2021-08-30|
|        101|     2|    20|   2021-09-01|
|        101|     3|    10|   2021-09-05|
|        102|     1|    50|   2021-09-02|
|        103|     1|    25|   2021-09-01|
+-----------+------+------+-------------+
only showing top 5 rows



In [15]:
result = df.withColumn(
    'key_col',
    F.concat('shipment_Id', 'sub_id')
    ).groupBy(
        F.concat(F.year('shipment_date'),F.lit('-'), F.month('shipment_date')).alias('year_month'),
        F.concat_ws('-', F.year('shipment_date'), F.month('shipment_date')).alias('year_month_1')
        ).agg(
            F.countDistinct('key_col').alias('count')
            ).show()

+----------+------------+-----+
|year_month|year_month_1|count|
+----------+------------+-----+
|    2021-9|      2021-9|    6|
|    2021-8|      2021-8|    3|
+----------+------------+-----+

